In [1]:
import pandas as pd

In [37]:
carts = pd.read_csv('data/Cart.csv')
orders = pd.read_csv('data/Order.csv')
orders_history = pd.read_csv('data/OrderHistory.csv')
entreprises = pd.read_csv('data/Entreprise.csv')

In [113]:
def get_entreprise_email(id_entreprise):
    entreprise_email = entreprises.loc[entreprises['id'] == id_entreprise]['email']
    index = entreprise_email.index[0]
    return entreprise_email[index]

def get_entreprise_name(id_entreprise):
    entreprise_name = entreprises.loc[entreprises['id'] == id_entreprise]['name']
    index = entreprise_name.index[0]
    return entreprise_name[index]

def get_entreprise_city(id_entreprise):
    entreprise_city = entreprises.loc[entreprises['id'] == id_entreprise]['city']
    index = entreprise_city.index[0]
    return entreprise_city[index]

def is_order_take(id_order):
    return (id_order in orders_history['id_order'])

def get_quantities(id_cart):
    quantity_reserved = orders.loc[orders['id_cart'] == id_cart]['quantity']
    quantity_take = 0
    for id_order in quantity_reserved.index:
        if is_order_take(id_order): quantity_take += quantity_reserved[id_order]
    quantity_reserved = quantity_reserved.sum()
    quantity_published = carts.loc[carts['id'] == id_cart]['quantity'] + quantity_reserved
    quantity_published = quantity_published.sum()
    return {
        'quantity_published': quantity_published,
        'quantity_reserved': quantity_reserved,
        'quantity_take': quantity_take
    }

In [114]:
dataset = list()
for index in carts.index:
    publication_date = pd.to_datetime(carts['creation_date'][index])
    quantities = get_quantities(carts['id'][index])
    row = {
        'cart_type': carts['type'][index],
        'entreprise_email': get_entreprise_email(carts['id_entreprise'][index]),
        'entreprise_name': get_entreprise_name(carts['id_entreprise'][index]),
        'entreprise_city': get_entreprise_city(carts['id_entreprise'][index]),
        'day_publication': publication_date.dayofweek,
        'month_publication': publication_date.month,
        'hour_publication': publication_date.hour,
        'quantity_published': quantities['quantity_published'],
        'quantity_reserved': quantities['quantity_reserved'],
        'quantity_take': quantities['quantity_take']
    }
    dataset.append(row)

In [117]:
import csv

def write_csv(filename, fieldnames, rows):
    file_path = f'data/{filename}'
    with open(file_path, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

write_csv("Prediction.csv", dataset[0].keys(), dataset)